In [1]:
!pip install --user -r requirements.txt

ERROR: Ignored the following versions that require a different python version: 1.2.0 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement contourpy==1.2.0 (from versions: 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.1.0, 1.1.1rc1, 1.1.1)
ERROR: No matching distribution found for contourpy==1.2.0


In [2]:
from scipy.signal import butter, filtfilt
from scipy import fft
import numpy as np
from wettbewerb import get_3montages, EEGDataset
from scipy import signal as sig
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [13]:
def butter_bandpass_filter(_signal, lowcut, highcut, freq, order=4):
    nyquist = 0.5 * freq
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_sig = filtfilt(b, a, _signal)
    return filtered_sig


def compute_band_power(sig, freq, freq_low, freq_high):
    spectrum = fft.fft(sig)
    frequencies = np.fft.fftfreq(len(sig), d=1/freq)
    freq_range = (frequencies >= freq_low) & (frequencies <= freq_high)
    power = np.sum(np.abs(spectrum[freq_range])**2) / len(freq_range)
    return power
    
    
def pre_processing(_channels, _data, fr):#, scaler):
    _montage, _montage_data, _is_missing = get_3montages(_channels, _data)

    amplitude_means = np.zeros(len(_montage))
    mean_amplitude = []
    amplitude_max = np.zeros(len(_montage))
    max_amplitude = []
    amplitude_std = np.zeros(len(_montage))
    std_amplitude = []
    
    band_power = np.zeros(4)

    for j, signal_name in enumerate(_montage):
        signal = _montage_data[j]
        signal_notch = mne.filter.notch_filter(x=signal, Fs=fr, freqs=np.array([50., 100.]), 
                                                n_jobs=2, verbose=False)
        signal_filter = butter_bandpass_filter(_signal=signal, lowcut=0.5, 
                                                   highcut=70.0, freq=fr)

        # Calculate FFT and mean amplitude
        spectrum = fft.fft(signal_filter)
        amplitude_means[j] = np.mean(np.abs(spectrum))
        amplitude_max[j] = np.max(np.abs(spectrum))
        amplitude_std[j] = np.std(np.abs(spectrum))

        # Compute band power
        band_power[0] += compute_band_power(signal_filter, fr, 0.5, 4)
        band_power[1] += compute_band_power(signal_filter, fr, 4, 8)
        band_power[2] += compute_band_power(signal_filter, fr, 8, 13)
        band_power[3] += compute_band_power(signal_filter, fr, 13, 30)
        
    mean_amplitude.append(np.mean(amplitude_means))
    mean_amplitude = np.array(mean_amplitude)
    max_amplitude.append(np.mean(amplitude_max))
    max_amplitude = np.array(max_amplitude)
    std_amplitude.append(np.mean(amplitude_std))
    std_amplitude = np.array(std_amplitude)
    
    band_power /= len(_montage)
    features = np.concatenate((mean_amplitude, max_amplitude, std_amplitude, band_power), axis=0)
    #features_reshaped = features.reshape(1, -1)
    #feature_scaled = scaler.transform(features_reshaped)
    return features

In [14]:
def pad_eeg_signals(eeg_signals, target_length, fs=256):
    """
    对不同长度的EEG信号进行填充，使它们长度相同。
    :param eeg_signals: 包含多个EEG信号的列表，每个信号可能长度不同。
    :param target_length: 目标长度（秒），所有信号将被填充或截断至此长度。
    :param fs: EEG数据的采样频率（Hz）。
    :return: 填充后的EEG信号列表。
    """
    padded_signals = []
    target_samples = target_length * fs  # 将目标长度转换为样本数

    for signal in eeg_signals:
        if len(signal) < target_samples:
            # 如果信号长度小于目标长度，则进行零填充
            padded_signal = np.pad(signal, (0, target_samples - len(signal)), 'constant')
        else:
            # 如果信号长度大于或等于目标长度，则截断至目标长度
            padded_signal = signal[:target_samples]
        padded_signals.append(padded_signal)
    padded_signals = np.array(padded_signals)
    
    return padded_signals


In [15]:
def extract_eeg_data():
    """
    load eeg signal and do padding, feature extraction one by one
    :return: all_window_features: features on each window of EEG signals, shape (6213,59,7), representing 6213 EEG data, each having 59 windows, each window 7 features
    :return: all_window_labels: the correspoing label on each window of eeg signals. shape(6213,59)
    :return: all_eeg_labels: the label of each. signal correspoing the the. number in REFERENCE.csv
    """
    #initilize channels, data, sampling_frequencies, reference_system, all_eeg_labels
    channels: List[List[str]] = []
    data: List[np.ndarray] = []
    sampling_frequencies: List[float] = []
    reference_systems: List[str] = []
    all_eeg_labels: List[Tuple[bool,float,float]] = []
    
    #List. to store features and labels on each window
    all_window_features = []
    all_window_labels = []
    #define a standard scaler
    scaler = StandardScaler()
    dataset = EEGDataset("../shared_data/training")
    #initilize a np array to store the features
    features = np.empty((0,59,7))
    
    for item in dataset:
        _eeg_label = item[5] 
        all_eeg_labels.append(item[5])
        _chann =item[1]
        _eeg_signals=item[2]
        _fs=item[3]
        # pad each signal eeg signal to lenth 300 with its own sampling frequency
        padded_signal = pad_eeg_signals(_eeg_signals, 300, _fs) 
        # scale the padded signal
        padded_signal = scaler.fit_transform(padded_signal)
        window_size = 10 # each window is 10 seconds
        step_size = 5 # step size. if 5 seconds
        
        window_samples = window_size * _fs # transoform window size from time to number of samples
        step_samples = step_size * _fs # transoform step size size from time to number of samples
        # initialize list to store the result
        all_windows=[]
        window_features = []
        window_labels = []
        
        for start_idx in range(0, padded_signal.shape[1] - window_samples + 1, step_samples):
            onset_sample_point = int(_eeg_label[1] * _fs) # define the onset to sample point
            offset_sample_point = int(_eeg_label[2] * _fs) if _eeg_label[2]<=300 else 300* _fs # define the offset to sample point
            
            window = padded_signal[:, start_idx:start_idx + window_samples] #window of padded signal
            window_feature = pre_processing(_chann, window, _fs) # extract hidden. features of this window
            # if the window is totally on the left of onset point or totally on the right of offset point,or the onset point is zero, label this window 0
            if (start_idx < onset_sample_point and start_idx + window_samples < onset_sample_point) or (start_idx>offset_sample_point and start_idx + window_samples > offset_sample_point) or onset_sample_point== 0:
                window_label = 0
            else:
                window_label = 1 #apart form the conditions, label the window 1
            window_labels.append(window_label)
            window_features.append(window_feature)
            all_windows.append(window)
        window_features = np.array(window_features)
        window_labels = np.array(window_labels)
        all_window_features.append(window_features)
        all_window_labels.append(window_labels)
    
    all_window_features = np.array(all_window_features)
    all_window_labels = np.array(all_window_labels)
        
    #features = np.vstack((features, new_feature))
    return all_window_features, all_window_labels, all_eeg_labels

In [3]:
# dataset of all features, shape (6213,59,7)
#representing 6213 EEG data, each having 59 windows, each window 7 features
all_window_features = np.load('all_window_features.npy') 
# all EEG data labels ,shape (6213,59),6213 EEG data, each of them has 59 labels of 0 or 1
all_window_labels = np.load('all_window_labels.npy')
# all EEG data labels, shape(6213,3), representing 6213 EEG data, each having 3 label, seizure-onset-offset
eeg_labels = np.load('eeg_labels.npy')
# label representing if the EEG signal is seizure
seizure = [item[0] for item in eeg_labels]
seizure = np.array(seizure)

In [4]:
print(eeg_labels.shape)
print(all_window_features.shape)
print(all_window_labels.shape)
print(seizure.shape)

(6213, 3)
(6213, 59, 7)
(6213, 59)
(6213,)


In [5]:
# find the first index of three continous window, whose label is 1
def find_first_triple_one(sequence):
    if len(sequence) < 3:  
        return 0

    window_sum = sequence[0] + sequence[1] + sequence[2]
    if window_sum == 3:
        return 0

    for i in range(3, len(sequence)):
        window_sum += sequence[i] - sequence[i-3] #add new element, delete the first one 
        if window_sum == 3:
            return i - 2  # return the first index of these three windows

    return 0 


In [6]:
# reshape the features, so that all the windows are mixed together
# label also
reshaped_features = all_window_features.reshape(-1, 7)
reshaped_labels = all_window_labels.reshape(-1)


In [7]:
X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test = train_test_split(all_window_features, eeg_labels, all_window_labels, test_size=0.2, random_state=42, stratify=seizure)

In [8]:
# train test split of mixed window features and labels
x_train, x_test, y_train, y_test = train_test_split(reshaped_features, reshaped_labels, random_state = 42)

In [9]:
# build random forest classifier
rf = RandomForestClassifier(random_state = 42)

# train RF with mixed window, cause in the following prediction part, we classify each window, instead of each eeg signal
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
# determine the onset of each signal
onset=[] 
for signal in X_test:
    window_seizure=[] # store the label of each window in a single signal
    for window in signal:
        _prediction = rf.predict(window.reshape(1, -1)) # predict the label of each window 
        window_seizure.append(_prediction[0]) # store the label(0 or 1) 
        first_index = find_first_triple_one(window_seizure)# find the first index of 3 continous window
    onset.append(first_index * 5) # step size is 5, onset is index * 5
print(onset)
        

[0, 35, 0, 0, 0, 0, 0, 0, 140, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 155, 0, 0, 0, 0, 0, 30, 0, 0, 0, 90, 0, 0, 0, 0, 0, 0, 0, 0, 190, 105, 0, 0, 0, 0, 0, 0, 65, 0, 0, 0, 145, 0, 0, 0, 0, 270, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 25, 0, 0, 60, 0, 0, 180, 0, 0, 0, 0, 0, 0, 0, 35, 0, 0, 0, 0, 20, 45, 0, 0, 0, 0, 20, 150, 0, 0, 0, 0, 0, 0, 80, 85, 185, 0, 0, 30, 15, 0, 0, 0, 0, 0, 0, 80, 0, 0, 115, 0, 20, 0, 0, 0, 80, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 200, 0, 0, 160, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 60, 0, 0, 0, 100, 0, 45, 0, 0, 0, 90, 0, 0, 140, 0, 0, 0, 0, 0, 30, 15, 155, 0, 55, 0, 0, 25, 0, 0, 20, 0, 0, 0, 0, 0, 5, 0, 15, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 90, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 135, 0, 0, 0, 0, 235, 0, 25, 0, 50, 0, 0, 25, 25, 0, 0, 0, 0, 0, 0, 70, 0, 0, 0, 0, 0, 60, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 40, 0, 75, 0, 0, 0, 10, 0, 

In [11]:
# determine the label of each signal
# if the onset of this signal is not zero, the label is 1, instead, 0

eeg_signal_label = []
for item in onset:
    if item>0:
        eeg_signal_label.append(1)
    else:
        eeg_signal_label.append(0)
        

In [12]:
Y1_test_label = [item[0] for item in Y1_test]
# confusion matrix and report
cm = confusion_matrix(Y1_test_label, eeg_signal_label)
print("Confusion Matrix:")
print(cm)

report = classification_report(Y1_test_label, eeg_signal_label)
print("Classification Report:")
print(report)

Confusion Matrix:
[[759   0]
 [215 269]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.78      1.00      0.88       759
         1.0       1.00      0.56      0.71       484

    accuracy                           0.83      1243
   macro avg       0.89      0.78      0.80      1243
weighted avg       0.87      0.83      0.81      1243



In [13]:
import joblib

joblib.dump(rf, 'random_forest_model.joblib')

['random_forest_model.joblib']